In [ ]:
import random
import numpy as np
import sys
from random import randint
from data_load import *
from cwsabie_inner import *
from transE_label import *
from gensim.corpora import Dictionary
from collections import Counter
from gensim.models import Word2Vec
%load_ext cython

# Wsabie

In [53]:
%%cython
import numpy as np

cimport numpy as np
from random import randint
import sys
import cython
cdef extern from "math.h":
    double sqrt(double m)
import math
from libc.stdlib cimport malloc, free

from libc.math cimport exp
from libc.math cimport log
from gensim.matutils import argsort

from libc.string cimport memset
import random
# scipy <= 0.15

import scipy.linalg.blas as fblas
ctypedef np.float32_t REAL_t
cdef int ONE = 1


REAL = np.float32
cdef extern from "/Users/mayk/working/figer/baseline/PLE/Model/warp/voidptr.h":
    void* PyCObject_AsVoidPtr(object obj)
DEF MAX_SENTENCE_LEN = 10000
ctypedef void (*scopy_ptr) (const int *N, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef void (*saxpy_ptr) (const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef float (*sdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*dsdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*snrm2_ptr) (const int *N, const float *X, const int *incX) nogil
ctypedef void (*sscal_ptr) (const int *N, const float *alpha, const float *X, const int *incX) nogil


cdef scopy_ptr scopy = <scopy_ptr>PyCObject_AsVoidPtr(fblas.scopy._cpointer)  # y = x
cdef saxpy_ptr saxpy=<saxpy_ptr>PyCObject_AsVoidPtr(fblas.saxpy._cpointer)  # y += alpha * x
cdef sdot_ptr sdot=<sdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # float = dot(x, y)
cdef dsdot_ptr dsdot=<dsdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # double = dot(x, y)
cdef snrm2_ptr snrm2=<snrm2_ptr>PyCObject_AsVoidPtr(fblas.snrm2._cpointer)  # sqrt(x^2)
cdef sscal_ptr sscal=<sscal_ptr>PyCObject_AsVoidPtr(fblas.sscal._cpointer) # x = alpha * x
DEF EXP_TABLE_SIZE = 10000
DEF MAX_EXP = 50

cdef REAL_t[EXP_TABLE_SIZE] EXP_TABLE
cdef REAL_t[EXP_TABLE_SIZE] LOG_TABLE

cdef REAL_t ONEF = <REAL_t>1.0

# for when fblas.sdot returns a double
cdef REAL_t our_dot_double(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>dsdot(N, X, incX, Y, incY)

# for when fblas.sdot returns a float
cdef REAL_t our_dot_float(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>sdot(N, X, incX, Y, incY)

# for when no blas available
cdef REAL_t our_dot_noblas(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    # not a true full dot()-implementation: just enough for our cases
    cdef int i
    cdef REAL_t a
    a = <REAL_t>0.0
    for i from 0 <= i < 50 by 1:
        a += X[i] * Y[i]
    return a

# for when no blas available
cdef void our_saxpy_noblas(const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil:
    cdef int i
    for i from 0 <= i < N[0] by 1:
        Y[i * (incY[0])] = (alpha[0]) * X[i * (incX[0])] + Y[i * (incY[0])]
cdef REAL_t cvdot(vec1,vec2,size):
    cdef int csize = size
    f= dsdot(&csize,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
def csaxpy(vec1,vec2,alpha,size):
    cdef int csize = size
    cdef float calpha = alpha
    f= our_saxpy_noblas(&csize,&calpha,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
cdef REAL_t crank(int k):
    cdef REAL_t loss = 0.
    cdef int i = 1
    for i in range(1,k+1):
        loss += ONEF/i
    return loss
cdef REAL_t vsum(REAL_t *vec,int *size):
    cdef int i
    cdef REAL_t product
    product = <REAL_t>0.0
    for i from 0 <= i < size[0] by 1:
        product += vec[i] **2
    return sqrt(product)
def cnorm(vec):
    cdef int size
    size  = len(vec)
    return vsum(<REAL_t *>(np.PyArray_DATA(vec)),&size)
def init():
    for i in range(EXP_TABLE_SIZE):
        EXP_TABLE[i] = <REAL_t>exp((i / <REAL_t>EXP_TABLE_SIZE * 2 - 1) * MAX_EXP)
        EXP_TABLE[i] = <REAL_t>(EXP_TABLE[i] / (EXP_TABLE[i] + 1))
#init()


def ctrain(As,B,insts,size,lr,gradient,it,Verbose=False):
    cdef float error
    next_random = 1
    error = 0.
   # cdef lam = 0.01
    
    for i,inst in enumerate(insts):
        err,next_random =gradient(As,B,inst,size,next_random,lr=lr)
      
        error += err
        if i % 1000 ==0 and Verbose:
            sys.stdout.write("\rIteration %d " % (it)+ "trained {0:.0f}%".format(float(i)*100/len(insts))+" Loss:{0:.2f}".format(error))
            sys.stdout.flush()
    if Verbose:
        sys.stdout.write("\n")
    return error

cdef void divide(REAL_t *vec, const float *alpha, const int *size):
    cdef int i
    for i from 0 <= i < size[0] by 1:
        vec[i] = vec[i]/alpha[0]
def cdivide(vec,alpha):
    cdef int size
    size  = len(vec)
    cdef float r = alpha
    divide(<REAL_t *>(np.PyArray_DATA(vec)),&r,&size)

    
def maximum(vec):
    maximum = -float('inf')
    idx = -1
    for i in range(len(vec)):
        if maximum < vec[i]:
            maximum = vec[i]
            idx = i
    return idx,maximum

def warp_gradient(As,B,inst,size,next_random,lr=0.01):
    #print B
    #print B[0]-B[9]
    cdef unsigned long long  c_next_random = next_random
    dA = np.zeros([As.shape[0],size],dtype=REAL)
    dB = np.zeros(B.shape,dtype=REAL)

    X = [np.sum(A[inst.features],axis=0) for A in As]
    cdef REAL_t error = 0.
    cdef REAL_t clr = lr
    cdef int N,n_sample 
    cdef int neg_num = len(inst.negative_labels)
    cdef REAL_t norm
    cdef int cSize = size
    cdef REAL_t floats
   # ranks = argsort(scores,reverse=True)
    M = len(inst.sparse_labels)
    for i,l in enumerate(inst.sparse_labels):
        max_idx,s1= maximum([cvdot(x,B[l],cSize) for x in X])
 
        N=1
        n_sample  = -1
        for k in range(neg_num):
            c_next_random = random_int32(&c_next_random)
            nl = inst.negative_labels[c_next_random%neg_num]#randint(0,neg_num-1)]
            neg_idx,s2 = maximum([cvdot(x,B[nl],cSize) for x in X])
            if s1 - s2<1:
                n_sample = nl
                N = k+1
                break
        if n_sample!=-1:

            L = crank(len(inst.negative_labels)/N)#*(crank(M/(ranks[i]+1)))
            negL = -L
            error += (1+s2-s1)*L

            csaxpy(B[l],dA[max_idx],L,cSize)
            csaxpy(-B[n_sample],dA[neg_idx],L,cSize)

    for f in inst.features:
        for i in range(As.shape[0]):
            csaxpy(dA[i],As[i][f],clr,cSize)
            norm = cnorm(As[i][f])
            if norm >1:
                cdivide(As[i][f],norm)

    
    
    return error,c_next_random
def save_to_text(matrix,output):
    shape = matrix.shape
    with open(output,'wb') as out:
        out.write("%d %d\n" % (shape))
        for row in matrix:
            x = " ".join(map(lambda x:"{0:.5}".format(x),row))
            out.write(x+"\n")

cdef inline unsigned long long random_int32(unsigned long long *next_random) nogil:
    next_random[0] = (next_random[0] * <unsigned long long>25214903917ULL + 11) & 281474976710655ULL
    return next_random[0]
def crand(sed):
    cdef unsigned long long csed = sed
    return random_int32(&csed)
def save2bin(mat,dct,fn):
    n,d  = mat.shape
    with open(fn,'w') as out:
        out.write("%d %d\n" % (n,d))
        for i in range(n):
            text = " ".join(map(str,mat[i]))
            out.write("%s %s\n" %(dct[i],text))
def normalize(mat):
    for v in mat:
        norm = np.linalg.norm(v)
        if norm >=1:
            v /= norm

In [13]:
in_dir= "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN"
a=MentionData('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_x_new.txt',
              "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_y.txt",
             in_dir+"/feature.txt",in_dir+"/type.txt")
label_bin  = Word2Vec.load_word2vec_format('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/labels.bin')



In [56]:
np.random.seed(12)
size= 300
K=2
A= np.random.uniform(-6/np.sqrt(size),6/np.sqrt(size), [K,len(a.feature2id),size]).astype(np.float32)#np.random.rand(len(a.feature2id),size).astype(np.float32)
B = np.asarray([label_bin[a.id2label[i]] for i in range(len(label_bin.vocab))],dtype=np.float32)#np.random.normal(0,1.0, [len(a.label2id),size]).astype(np.float32)
next_random = 1
normalize(A)
normalize(B)

for i in range(15):
    reg_err =0.
    #B-=0.15*(B-C)
    ctrain(A,B,a.data,size,0.001,warp_gradient,it=i,Verbose=True)
for i in range(K):
    save2bin(A[i],a.id2feature,"/Users/mayk/working/figer/baseline/PLE/Results/BBN/warp_As/warp_A_%d.bin" % (i))

save2bin(B,a.id2label,'/Users/mayk/working/figer/baseline/PLE/Results/BBN/warp_B.bin')

Iteration 0 trained 100% Loss:253125.52
Iteration 1 trained 100% Loss:178785.86
Iteration 2 trained 100% Loss:155965.55
Iteration 3 trained 100% Loss:143846.16
Iteration 4 trained 100% Loss:135390.00
Iteration 5 trained 100% Loss:127930.06
Iteration 6 trained 100% Loss:122679.48
Iteration 7 trained 100% Loss:117701.02
Iteration 8 trained 100% Loss:112958.53
Iteration 9 trained 100% Loss:110220.31
Iteration 10 trained 100% Loss:106399.84
Iteration 11 trained 100% Loss:103590.56
Iteration 12 trained 100% Loss:100810.35
Iteration 13 trained 100% Loss:98670.17
Iteration 14 trained 100% Loss:96363.98


In [43]:
A= np.random.uniform(-6/np.sqrt(size),6/np.sqrt(size), [K,len(a.feature2id),size]).astype(np.float32)#np.random.rand(len(a.feature2id),size).astype(np.float32)

array([[ 0.05660552, -0.28167629, -0.20095788, ..., -0.04172279,
         0.24144581,  0.26826933],
       [ 0.01179722, -0.16205899, -0.00426593, ..., -0.10529076,
         0.245049  , -0.17339016],
       [ 0.23352301, -0.18039671, -0.10599529, ..., -0.15253411,
         0.29050085, -0.02163795],
       ..., 
       [ 0.32330236, -0.03868889,  0.18012173, ...,  0.05494594,
         0.21774139, -0.29889831],
       [ 0.1789182 , -0.23977904, -0.3417342 , ..., -0.21547624,
         0.19111684, -0.19578032],
       [-0.01055121,  0.18243976,  0.04139563, ..., -0.26376632,
        -0.15031138, -0.13791315]], dtype=float32)

array([[ 0.32568955,  0.3092328 , -0.30828238, ...,  0.00127495,
        -0.22513478,  0.26290956],
       [ 0.19257526,  0.32855147, -0.34415814, ...,  0.00249389,
        -0.22354276,  0.22557129],
       [ 0.16040505, -0.30075097, -0.33582723, ...,  0.03331999,
        -0.02405227, -0.32290831],
       ..., 
       [-0.11637408,  0.30286971,  0.28565001, ...,  0.0366358 ,
        -0.00080049,  0.30402589],
       [ 0.11059088, -0.23062417,  0.04827109, ...,  0.0069234 ,
         0.05998977,  0.2990427 ],
       [ 0.29318094,  0.17393352,  0.17989096, ...,  0.19187552,
         0.18237527, -0.14068848]], dtype=float32)